In [ ]:
Please do same exercise using two different framework.

Framework 1. pandas
framework 2. apache beam python https://beam.apache.org/documentation/sdks/python/


using two input files dataset1 and dataset2 

join dataset1 with dataset2 and get tier

generate below output file

legal_entity, counterparty, tier, max(rating by counterparty), sum(value where status=ARAP), sum(value where status=ACCR)

Also create new record to add total for each of legal entity, counterparty & tier.

Sample data:
legal_entity, counterparty, tier, max(rating by counterparty), sum(value where status=ARAP), sum(value where status=ACCR)
L1,Total, Total, calculated_value, calculated_value,calculated_value
L1, C1, Total,calculated_value, calculated_value,calculated_value
Total,C1,Total,calculated_value, calculated_value,calculated_value
Total,Total,1,calculated_value, calculated_value,calculated_value
L2,Total,Total,calculated_value, calculated_value,calculated_value
....
like all other values.

where caluclated_value in sample data needs to be calculated using above method.

In [3]:
import pandas as pd

# Load the datasets into Pandas DataFrames
dataset1 = pd.read_csv('../input_data/dataset1.csv')
dataset2 = pd.read_csv('../input_data/dataset2.csv')

# Merge the datasets on the 'counterparty' column
merged_data = pd.merge(dataset1, dataset2, on='counter_party')


In [4]:
# Create new columns for the sums based on the 'status' column
merged_data['value_ARAP'] = merged_data.apply(lambda row: row['value'] if row['status'] == 'ARAP' else 0, axis=1)
merged_data['value_ACCR'] = merged_data.apply(lambda row: row['value'] if row['status'] == 'ACCR' else 0, axis=1)

In [7]:
# Perform the necessary calculations
result = merged_data.groupby(['legal_entity', 'counter_party', 'tier']).agg({
    'rating': 'max',
    'value_ARAP': 'sum',
    'value_ACCR': 'sum'
}).reset_index()

In [9]:
# Add the total rows
total = result.groupby(['legal_entity', 'counter_party']).agg({
    'tier': 'count',
    'rating': 'max',
    'value_ARAP': 'sum',
    'value_ACCR': 'sum'
}).reset_index()

In [10]:
total['tier'] = 'Total'

In [11]:
result = pd.concat([result, total], ignore_index=True)

In [12]:
result.to_csv('output.csv', index=False)


In [13]:
result

,legal_entity,counter_party,tier,rating,value_ARAP,value_ACCR
0,L1,C1,1,3,40,0
1,L1,C3,3,6,5,0
2,L1,C4,4,6,40,100
3,L2,C2,2,3,20,40
4,L2,C3,3,2,0,52
5,L2,C5,5,6,1000,115
6,L3,C3,3,4,0,145
7,L3,C6,6,6,145,60
8,L1,C1,Total,3,40,0
9,L1,C3,Total,6,5,0


In [ ]:
# Perform the necessary calculations
result = merged_data.groupby(['legal_entity', 'counter_party', 'tier']).agg({
    'rating': 'max',
    'value': ['sum' if x == 'ARAP' else 0 for x in merged_data['status']],
    'value': ['sum' if x == 'ACCR' else 0 for x in merged_data['status']]
}).reset_index()

In [ ]:
# Add the total rows
total = result.groupby(['legal_entity', 'counter_party']).agg({
    'tier': 'count',
    'rating': 'max',
    'value': 'sum'
}).reset_index()

In [2]:




total['tier'] = 'Total'
result = pd.concat([result, total], ignore_index=True)

# Save the output DataFrame as a CSV file
result.to_csv('output.csv', index=False)


/var/folders/_l/g109827s4sv_6p83rwst2h7h0000gp/T/ipykernel_4121/1510949160.py:11: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  result = merged_data.groupby(['legal_entity', 'counter_party', 'tier']).agg({


TypeError: 'int' object is not callable